In [31]:
import cv2
import numpy as np
import pandas as pd

N = ord('B') % 5 + 1
N

2

In [35]:
df = pd.read_excel('lab6.xlsx')
df.loc[N, 'file name']

'draco.jpg'

In [27]:
# Load Haar cascades for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Read the image
image_path = './images/emma-watson.jpg'
image = cv2.imread(image_path)

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

# Get lens size factor with input validation
while True:
    try:
        lens_size_factor = float(input("Enter lens factor (0.1 - 0.5): "))
        if 0.1 <= lens_size_factor <= 0.5:
            break
        else:
            print("Please enter a value between 0.1 and 0.5.")
    except ValueError:
        print("Invalid input. Please enter a numeric value between 0.1 and 0.5.")

# Loop through each detected face
for (x, y, w, h) in faces:
    # Region of interest (ROI) for the face
    roi_gray = gray[y:y + h, x:x + w]
    roi_color = image[y:y + h, x:x + w]

    # Detect eyes in the face region
    eyes = eye_cascade.detectMultiScale(roi_gray)

    # If at least two eyes are detected
    if len(eyes) >= 2:
        # Sort the eyes by size (area)
        eyes = sorted(eyes, key=lambda e: e[2]*e[3], reverse=True)
        eye1 = eyes[0]
        eye2 = eyes[1]

        # Calculate the centers of the eyes
        center1 = (x + eye1[0] + eye1[2] // 2, y + eye1[1] + eye1[3] // 2)
        center2 = (x + eye2[0] + eye2[2] // 2, y + eye2[1] + eye2[3] // 2)

        # Calculate the distance between the eyes to determine the glasses size
        dx = center2[0] - center1[0]
        dy = center2[1] - center1[1]
        dist = np.sqrt(dx**2 + dy**2)
        radius = int(dist * (lens_size_factor))

        # Glasses color (black)
        color = (255, 0, 0)

        # Draw circles on the eyes
        cv2.circle(image, center1, radius, color, 2)
        cv2.circle(image, center2, radius, color, 2)

        # Draw the bridge between the lenses
        point1 = (center1[0] + radius, center1[1])
        point2 = (center2[0] - radius, center2[1])
        cv2.line(image, point1, point2, color, 2)

        # Draw the arms of the glasses
        # Left arm
        cv2.line(image, (center1[0] - radius, center1[1]),
                 (center1[0] - int(radius * 1.5), center1[1] - int(radius * 0.5)), color, 2)
        # Right arm
        cv2.line(image, (center2[0] + radius, center2[1]),
                 (center2[0] + int(radius * 1.5), center2[1] - int(radius * 0.5)), color, 2)

        # After processing one face, exit the loop
        break

# Save the output image
cv2.imwrite("output.jpg", image)

Enter lens factor (0.1 - 0.5):  0.33


True